<a href="https://colab.research.google.com/github/Abu-Kowcher-Rmstu/machine_learning_lab/blob/main/multi_disease_prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

In [19]:
df=pd.read_csv("/content/drive/MyDrive/ColabNotebooks/multi disease prediction/diabetes2.csv")

In [20]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [21]:
X = df.drop(columns = 'Outcome', axis=1)
y = df['Outcome']

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20,random_state = 2022,stratify=y)

In [23]:
from sklearn.metrics import classification_report,ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
def clac(y_test, y_pred, classifier):
  from sklearn.metrics import confusion_matrix
 
  print("Classifier name: ", classifier)
  print(classification_report(y_test, y_pred))

  #cm = confusion_matrix(y_pred.round(),y_test)
  cm = confusion_matrix(y_test,y_pred)
  cmd_obj = ConfusionMatrixDisplay(cm)
  cmd_obj.plot()


In [24]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes' :{
        'model': MultinomialNB(),
        'params':{
        }
    },

    'Decision_tree': {
        'model' : DecisionTreeClassifier(random_state =10),
        'params': { 
            'criterion': ['entropy','gini']
        }
    },

    'KNN': {
        'model' : KNeighborsClassifier(metric='minkowski',n_neighbors = 1),
        'params': { 
            #'n_neighbors': list(range(1,15,2)),
            'p': [1,2]
        }
    }
    
    
}


In [25]:
from sklearn.model_selection import GridSearchCV

In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=10, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as detailed_score

precision,recall,fscore,support=detailed_score(y_test,y_pred,average='macro')
result = "%0.2f %0.2f %0.2f " % (precision, recall, fscore)
result

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support as detailed_score

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression(random_state=0)))
models.append(('KNN', KNeighborsClassifier( n_neighbors =1, metric = 'minkowski', p=2)))
models.append(('DT', DecisionTreeClassifier(criterion = 'entropy', random_state = 0)))
models.append(('RF', RandomForestClassifier(n_estimators=10, criterion ='entropy', random_state = 0)))
models.append(('MNB', MultinomialNB()))
models.append(('SVM', SVC(kernel = 'linear',probability=True, random_state = 0)))
models.append(('SGD', SGDClassifier(loss="log", penalty="l2", max_iter=1000)))

# evaluate each model in turn
scores =[] 
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	#kfold = model_selection.KFold(n_splits=10, random_state=seed)
  cv_results = cross_val_score(model, X_train, y_train, cv=10, scoring=scoring)
  #print("training:{}".format(model))
  #print("CV score: {:0.2f}".format(cv_results.mean()))
  model.fit(X_train,y_train)
  test_accuracy = model.score(X_test,y_test)
  y_pred = model.predict(X_test)
  precision,recall,fscore,support=detailed_score(y_test,y_pred,average='macro')
  results.append(cv_results)
  names.append(name)
  msg = "%s: %0.2f (%f)" % (name, cv_results.mean(), cv_results.std())
  #msg = "%s: %0.2f"% (name,test_accuracy)
  print(msg)
  print("test accuracy: {:0.2f}".format(test_accuracy))
  scores.append( 
      {
          'model': name,
          'cv_score': "%0.2f" %(cv_results.mean()),
          'test_score': "%0.2f" %(test_accuracy),
          'precision': "%0.2f" %(precision),
          'recall': "%0.2f" %(recall),
          'f1_score': "%0.2f" %(fscore),
             
      }
      
      )
# boxplot algorithm comparison
#plt.figure(figsize=(20,10))
fig = plt.figure(figsize=(10,7))
plt.title('Cross validation performance for Tri-gram feature')
plt.xlabel('Algorithms')
plt.ylabel('Accuracy')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()


#plt.savefig('result.png')


In [ ]:
output_df = pd.DataFrame(scores,columns=['model','cv_score','test_score','precision','recall','f1_score'])

In [ ]:
print("Results of TF-IDF(uni,bi and trigram approach)with all features and  removing stopwords")
output_df